# Code fourni : GAN Fully Connected sur Mnist

# CC - Martin BAUS

In [1]:
from __future__ import print_function
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import matplotlib.pyplot as plt

#cudnn.benchmark = True

#set manual seed to a constant get a consistent output
manualSeed = random.randint(1, 10000)
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  4311


## Modification du data loader

* Le dataloader ci-dessous a été modifié pour retourner des batchs d'exemples avec leurs labels sous forme de vecteurs on-hot-encodings

* Pour le vérifier vous pouvez taper la commande suivante et regarder les tailles de inputs, labels et le contenu de labels :

  `inputs, labels = next(iter(dataloader))`



In [4]:
#loading the dataset
from torchvision.transforms import Lambda


dataset = dset.MNIST(root='./data', download=True,
                       transform=transforms.Compose([
                           transforms.Resize(28),
                           transforms.ToTensor(),
                           transforms.Normalize((0.5,), (0.5,)),
                       ]), target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)) )
#number of channels in image(since the image is grayscale the number of channels are 1)
nc=1

dataloader = torch.utils.data.DataLoader(dataset, batch_size=64,
                                         shuffle=True, num_workers=2)

#checking the availability of cuda devices
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [6]:
inputs, labels = next(iter(dataloader))

PicklingError: Can't pickle <function <lambda> at 0x00000288DB665E40>: attribute lookup <lambda> on __main__ failed

In [7]:
nz = 50

In [ ]:
class GeneratorLinear(nn.Module):
    def __init__(self, nz=50, nh1=128, nh2=256, noutput=784):
        super(GeneratorLinear, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.Linear(nz,nh1),
            nn.ReLU(),
            nn.Linear(nh1,nh2),
            nn.ReLU(),
            nn.Linear(nh2,noutput),
            nn.Tanh()
        )
    def forward(self, input):
        output = self.main(input)
        return output

In [ ]:
class DiscriminatorLinear(nn.Module):
  def __init__(self, ninputs=784, nh1=128, nh2= 64 ):
        super(DiscriminatorLinear, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(ninputs,nh1),
            nn.ReLU(),
            nn.Linear(nh1, nh2),
            nn.ReLU(),
            nn.Linear(nh2,1),
            nn.Sigmoid()
        )

  def forward(self, input):
    output = self.main(input)
    return output.view(-1, 1).squeeze(1)

In [8]:
netG = GeneratorLinear().to(device)
print(netG)
netD = DiscriminatorLinear().to(device)
print(netD)

criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=0.002, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.002, betas=(0.5, 0.999))

GeneratorLinear(
  (main): Sequential(
    (0): Linear(in_features=60, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=784, bias=True)
    (5): Tanh()
  )
)
DiscriminatorLinear(
  (main): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [10]:
fixed_noise = torch.randn(64, nz, device=device)

real_label = 1
fake_label = 0

niter = 25

# Commented out IPython magic to ensure Python compatibility.
for epoch in range(niter):
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        netD.zero_grad()
        real_cpu = data[0].view([-1,784]).to(device)
        batch_size = real_cpu.size(0)
        label = torch.full((batch_size,), real_label, device=device, dtype=torch.float)

        output = netD(real_cpu)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        noise = torch.randn(batch_size, nz, device=device)
        fake = netG(noise)
        #label.fill_(fake_label)
        label = torch.full((batch_size,), fake_label, device=device, dtype=torch.float)

        output = netD(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()
        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()
        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
                   % (epoch, niter, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        if i % 100 == 0:
            fake = netG(fixed_noise)
            fake_images_np = fake.cpu().detach().numpy()
            fake_images_np = fake_images_np.reshape(fake_images_np.shape[0], 28, 28)
            R, C = 5, 5
            num_images_to_plot = min(batch_size, R * C)  # Limit images to plot to grid size
            for i in range(num_images_to_plot):
                plt.subplot(R, C, i + 1)
                plt.imshow(fake_images_np[i], cmap='gray')
            plt.show()

PicklingError: Can't pickle <function <lambda> at 0x00000288DB665E40>: attribute lookup <lambda> on __main__ failed

# 1. Implémentation d'un CGAN

On vous demande d'implémenter un CGAN en partant du code ci dessus (pas du votre ni d'un code récupéré sur internet). **Le code demandé doit être très proche du code fourni dans ce notebook pour le GAN.** Cela siognifie que vous ne devez modifier que ce qui doit être modifié dans le code du GAN.

On vous rappelle le critère d'optimisation d'un CGAN :

$$\min_{\theta_g} \max_{\theta_d} v(\theta_g, \theta_d) = \mathbb{E}_{x,y \sim p_{data}} [\log d(x,y)] +\mathbb{E}_{z\sim p_z, y' \sim p_y} [log(1-d(g(z,y'), y')]$$

où le générateur comme le discriminateur prenne en entrée le y, en plus du z (pour le générateur), en plus du x pour le discriminateur. Dans un CGAN le générateur comme le classifieur doivent prendre en entrée la condition de la génération. Ici vous choisirez d'utiliser la classe de l'exemple.


---



# Questions

Vous devez dans la suite écrire la classe du générateur, du discriminateur et la fonction de train pour optimiser le critère ci-dessus du CGAN.

Vous utiliserez un on-hot-encoding pour représenter les y (en dimension 10 donc) en entrée du classifieur et du discriminateur.   




## Aides

1. Voici ci-dessous un exemple de comment modifier le générateur pour prendre en entrée des couples (z,y) et produire une sortie qui dépend de ces deux entrées.

```
class GeneratorLinear(nn.Module):
    def __init__(self, nz=50, nh1=128, nh2=256, noutput=784):
        super(GeneratorLinear, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.Linear(nz+10,nh1),
            nn.ReLU(),
            nn.Linear(nh1,nh2),
            nn.ReLU(),
            nn.Linear(nh2,noutput),
            nn.Tanh()
        )
    def forward(self, input1, input2):
        input = torch.cat((input1,input2),1)
        output = self.main(input)
        return output
```

In [2]:
#On modifie le génerateur et Discriminateur pour qu'ils prennent en compte les 2 entrées

class GeneratorLinear(nn.Module):
    def __init__(self, nz=50, nh1=128, nh2=256, noutput=784):
        super(GeneratorLinear, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.Linear(nz+10,nh1),
            nn.ReLU(),
            nn.Linear(nh1,nh2),
            nn.ReLU(),
            nn.Linear(nh2,noutput),
            nn.Tanh()
        )
    def forward(self, input1, input2):
        input = torch.cat((input1,input2),1)
        output = self.main(input)
        return output

In [3]:
class DiscriminatorLinear(nn.Module):
  def __init__(self, ninputs=784, nh1=128, nh2= 64 ):
        super(DiscriminatorLinear, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(ninputs,nh1),
            nn.ReLU(),
            nn.Linear(nh1, nh2),
            nn.ReLU(),
            nn.Linear(nh2,1),
            nn.Sigmoid()
        )

  def forward(self, input1, input2):
    input = torch.cat((input1,input2),1)
    output = self.main(input)
    return output.view(-1, 1).squeeze(1)

2. Vous pouvez à chaque itération générer des exemples avec le CVGAN en utilisant des fixed_noise comme pour le GAN et des fixed_inputs pour les y. Cela vous aidera à voir si le modèle apprend à générer les bons chiffres pendant l'apprentissage. Vous pouvez utiliser par exemple :

  `fixed_input = next(iter(dataloader))[1][:64,:] # où 64 est le batch_size`

3. Vous pouvez générer des vecteurs aléatoires de one-hot-encoding à 10 classes avec la ligne suivante:
  
  `input2 = one_hot(torch.randint(0, 9, (batch_size,)),num_classes=10)`



In [ ]:
#1: Implémentation d'un CGAN
#On reprend le code d'un GAN 

fixed_noise = torch.randn(64, nz, device=device)

real_label = 1
fake_label = 0

niter = 25

# Commented out IPython magic to ensure Python compatibility.
for epoch in range(niter):
    fixed_input = next(iter(dataloader))[1][:64,:] #test pour fixer les yrand pour chaque itération
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x,y)) + log(1 - D(G(z,y),y))
        ###########################
        # train with real
        netD.zero_grad()
        real_cpu = data[0].view([-1,784]).to(device)
        batch_size = real_cpu.size(0)
        label = torch.full((batch_size,fixed_input), real_label, device=device, dtype=torch.float)

        output = netD(real_cpu)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        noise = torch.randn(batch_size, nz, device=device)
        fake = netG(noise)
        #label.fill_(fake_label)
        label = torch.full((batch_size,), fake_label, device=device, dtype=torch.float)

        output = netD(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()
        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()
        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
                   % (epoch, niter, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        if i % 100 == 0:
            fake = netG(fixed_noise)
            fake_images_np = fake.cpu().detach().numpy()
            fake_images_np = fake_images_np.reshape(fake_images_np.shape[0], 28, 28)
            R, C = 5, 5
            num_images_to_plot = min(batch_size, R * C)  # Limit images to plot to grid size
            for i in range(num_images_to_plot):
                plt.subplot(R, C, i + 1)
                plt.imshow(fake_images_np[i], cmap='gray')
            plt.show()

# 2. Génération d'images "dans le style" d'une image donnée MNIST

Une fois le CGAN  appris, utilisez le pour générer des images $i$ avec des $z$ et des $y$ variés. Le $y$ conditionne la classe de l'image générée et $z$ représente le reste de l'information du tracé (image penchée, trait épais etc). Collectez les triplets $(z,y,i)$.

Cela vous permet de constituer une base d'apprentissage pour apprendre un modèle $F$ à retrouver le $z$ à partir d'une image (d'une certaine façon vous apprenez a ionverser le Générateur): Ce modèle prend en entrée une image $i$ et prédit en sortie le $z$ correspondant. L'apprentissage peut être réalisé par minimisation du critère MSE (c'est une tâche de regression).

Une fois l'apprentissage du modèle $F$ réalisé vous pouvez l'utiliser pour générer des images de différents chiffre "dans le style" d'une image MNIST donnée, disons $i$ de la façon suivante. CHoisissez une image MNIST au hasard. Utilisez $F(i)$ pour prédire un vecteur $z$ corrsondant à l'image. Puis utilisez $F(i)$ en entrée du générateur conditionnel avec différentes vecteurs $y$ pour générer des chiffres "dans le style" de l'image $i$.

Réalisez les implémentations demandées